## Introduction

This notebook describes the analysis of extracellular recording data from Loren Frank Lab. The data was downloaded from CRNS website 

In [ ]:
import numpy as np
from numpy.fft import rfft
import scipy.io
from scipy import signal
from neurodsp.spectral import compute_spectrum, rotate_powerlaw

# Import utilities for loading and plotting data
from neurodsp.utils import create_times
from neurodsp.utils.download import load_ndsp_data
from neurodsp.plts.spectral import plot_power_spectra
from neurodsp.plts.time_series import plot_time_series
from neurodsp.filt import filter_signal

import matplotlib.pyplot as plt
%matplotlib qt